In [1]:
import pandas as pd
import re
from datetime import date 
from bs4 import BeautifulSoup
import requests
import time 
from requests_html import HTMLSession
import json
import random
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

# Amazon

- Need to download docker for scraping, please follow this video: https://www.youtube.com/watch?v=8q2K41QC2nQ&t=269s

In [16]:
def amazon_source_site(title):
    countries = {
        "com": "US",
        "es": "Spain",
        "it": "Italy",
        "co.uk": "UK",
        "de": "Germany",
        "fr": "France"
    }
    for code, country in countries.items():
        if f"Amazon.{code}" in title:
            return country
    return "Unknown"

In [17]:
def get_soup(url):   
    r = requests.get('http://localhost:8050/render.html', params={'url': url, 'wait': 2})  
    soup = BeautifulSoup(r.text, 'html.parser')  
    return soup  
  
def amazon_review(soup, url):    
    title = soup.title.text
    # print(title)
    amazon_site = amazon_source_site(title)
    # print(amazon_site)
    review = {}
    extracted_reviews = []   
    try:
        model = title.replace("Amazon.com: Customer reviews: ", "") \
                    .replace("Amazon.co.uk:Customer reviews: " if amazon_site == 'UK' else "", "") \
                    .replace("Amazon.de:Customer Reviews: " if amazon_site == 'Germany' else "", "") \
                    .replace("Amazon.it:Recensioni clienti: " if amazon_site == 'Italy' else "", "") \
                    .replace("Amazon.es:Opiniones de clientes: " if amazon_site == 'Spain' else "", "") \
                    .replace("Amazon.fr\xa0:Commentaires en ligne: " if amazon_site == 'France' else "", "").strip()
    except AttributeError: 
        try:
            model = soup.find("a", attrs={"data-hook": "product-link"})
        except AttributeError: 
            model = soup.find("h1", attrs={"class": "a-size-large a-text-ellipsis"}).string.strip()  
    # print('title:', model)
    try:
        Review_average = soup.find("i", attrs={'class':'a-icon a-icon-star-medium a-star-medium-4-5 averageStarRating'}).string.strip().replace(' out of 5 stars', '') \
                                .replace(' su 5 stelle' if amazon_site == 'Italy' else "", "") \
                                .replace(' de 5 estrellas' if amazon_site == 'Spain' else "", "") \
                                .replace(' sur 5\xa0étoiles' if amazon_site == 'France' else "", "").replace(',', '.')
    except AttributeError:
        Review_average = soup.find("span", attrs={'class':'a-icon-alt'}).string.replace(' out of 5 stars', '') \
                                .replace(' su 5 stelle' if amazon_site == 'Italy' else "", "") \
                                .replace(' de 5 estrellas' if amazon_site == 'Spain' else "", "") \
                                .replace(' sur 5\xa0étoiles' if amazon_site == 'France' else "", "").replace(',', '.')
    # print(Review_average)
    reviews = soup.find_all("div", {"data-hook": "review"})

    # Dictionary to map the site to the prefix
    review_prefixes_and_delimiters = {
        'Spain': ('Revisado en', ' el '),
        'Italy': ('Recensito in', ' il '),
        'France': ('Commenté en', ' le '),
        'UK': ('Reviewed in', ' on '),
        'US': ('Reviewed in', ' on '),
        'Germany': ('Reviewed in', ' on ')
    }
    if reviews:
        for item in reviews:    
            review_details = item.find('span', {'data-hook': 'review-date'}).text
            prefix, delimiter = review_prefixes_and_delimiters.get(amazon_site, ('Reviewed in', ' on '))
            review_details = review_details.replace(prefix, '').strip()
            
            if delimiter in review_details:
                date_country = review_details.split(delimiter)
                review_country = date_country[0].strip()
                review_date = date_country[1].strip()
            else:
                review_country = amazon_site  # Fallback to amazon_site
                review_date = review_details.strip()
            
            review = {
                'Amazon site': amazon_site,
                'Model': model,    
                'Review average' : Review_average,
                'Review date': review_date,    
                "Review content": item.find("span", {'data-hook': "review-body"}).text.strip(),  
                "Review country": review_country,
                "URL": url
            }
            
            try:    
                review["Review rating"] = float(item.find("i", {"data-hook": "review-star-rating"}).text.replace("out of 5 stars", "") \
                                                .replace(' su 5 stelle' if amazon_site == 'Italy' else "", "") \
                                                .replace(' de 5 estrellas' if amazon_site == 'Spain' else "", "") \
                                                .replace(' sur 5\xa0étoiles' if amazon_site == 'France' else "", "").replace(',', '.').strip())    
            except AttributeError:    
                review["Review rating"] = float(item.find("span", {"class": "a-icon-alt"}).text.replace("out of 5 stars", "") \
                                                .replace(' su 5 stelle' if amazon_site == 'Italy' else "", "") \
                                                .replace(' de 5 estrellas' if amazon_site == 'Spain' else "", "") \
                                                .replace(' sur 5\xa0étoiles' if amazon_site == 'France' else "", "").replace(',', '.').strip())    
            # print(review["Review rating"])
            try:    
                review['Review title']  = item.find("a", {'data-hook': "review-title"}).text.strip()    
            except AttributeError:    
                review['Review title']  = item.find("span", {'data-hook': "review-title"}).text.strip()    
            # print(review['Review title'])
      
            # try:    
            #     review["Verified Purchase or not"] = item.find("span", {'data-hook': "avp-badge"}).text.strip()    
            # except AttributeError:    
            #     review["Verified Purchase or not"] = None    
            # print(review["Verified Purchase or not"])
      
            # try:      
            #     review["Review name"] = item.find("span", {'class': "a-profile-name"}).string.strip()  
            # except AttributeError:        
            #     review["Review name"] = None  
      
            # try:    
            #     review["People find helpful"] = item.find("span", {'data-hook': "helpful-vote-statement"}).text.replace("people found this helpful", "").strip()    
            # except AttributeError:    
            #     review["People find helpful"] = None  
                
            # try:
            #     review['Seeding or not'] = item.find("span", {'class': "a-size-mini a-color-link c7yBadgeAUI c7yTopDownDashedStrike c7y-badge-text a-text-normal c7y-badge-link c7y-badge-vine-voice a-text-bold"}).text.strip() 
            # except AttributeError:   
            #     review['Seeding or not'] = None      
            extracted_reviews.append(review)    
        # print(review)
    else:
        review = {    
                'Amazon site': amazon_site,
                'Model': model,    
                'Review average' : Review_average,
                "URL" : url  
            }
        extracted_reviews.append(review)  
  
    return extracted_reviews 

## Links

In [18]:
# Read url
path =  r"Review text scrape URL.xlsx"
sheets = 'Amazon'
amazon_url = pd.read_excel(path, sheet_name = sheets)
urls = amazon_url['URL'].to_list()
url_list = []
for value in urls:
    if pd.notna(value):
        url_list.append(value)
url_list


['https://www.amazon.com/HP-936-Cyan-Original-Cartridge/product-reviews/B0CJCD2TWT',
 'https://www.amazon.com/HP-936-Magenta-Original-Cartridge/product-reviews/B0CJCDSS9Q',
 'https://www.amazon.com/HP-936-Yellow-Original-Cartridge/product-reviews/B0CJCDRYDD',
 'https://www.amazon.com/HP-936-Black-Original-Cartridge/product-reviews/B0CJCDJ88M',
 'https://www.amazon.com/product-reviews/B0CJCG8QKB/',
 'https://www.amazon.com/HP-936e-Cyan-EvoMore-Cartridge/product-reviews/B0CRD36HPR',
 'https://www.amazon.com/HP-936e-Magenta-EvoMore-Cartridge/product-reviews/B0CRD5DLVJ',
 'https://www.amazon.com/HP-936e-Yellow-EvoMore-Cartridge/product-reviews/B0CRD9J24L',
 'https://www.amazon.com/HP-936e-Black-EvoMore-Cartridge/product-reviews/B0CRCW5FGN',
 'https://www.amazon.co.uk/product-reviews/B0CRCHD5D5',
 'https://www.amazon.co.uk/product-reviews/B0CRCP133B',
 'https://www.amazon.co.uk/product-reviews/B0CRC61YBM',
 'https://www.amazon.co.uk/product-reviews/B0CRCCHBD3',
 'https://www.amazon.co.uk/pr

## Scrape Reviews
- Only last 10 pages of most recent reviews are retrieved

In [ ]:
start_time = time.time() 
  

star = ['one','two','three','four','five'] 
all_reviews = []  

for link in url_list:    
    print(link)  
    for y in star:  
        should_continue = True  
        for x in range(1, 11):  
            if not should_continue:  
                break   
            while True:
                # while True:
                try:  
                    # url = f'{link}/ref=cm_cr_getr_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber={x}&filterByStar={y}_star'    
                    # url = f'{link}/ref=cm_cr_arp_d_viewopt_sr?filterByStar={y}_star&pageNumber={x}'
                    # sort by Most recent reviews
                    url = f'{link}/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&filterByStar={y}_star&pageNumber={x}&sortBy=recent'
                    soup = get_soup(url)    
    
                    # print(f'Getting page: {x}')    
                    extracted_reviews = amazon_review(soup, url)   
                    print(f'Extracted reviews on page {x}: {len(extracted_reviews)},{y}star')
                    
                    all_reviews.extend(extracted_reviews)   

                    next_page_link = soup.find('li', {'class': 'a-last'})
    
                    if next_page_link is None or soup.find('li', {'class': 'a-disabled a-last'}):
                        should_continue = False
                        # print('No more pages left')
                        break 
                    else:   
                        break

                except Exception as e:  
                    print(f"Error encountered: {e}. Retrying in 3 seconds...")  
                    time.sleep(10)  

end_time = time.time()  

elapsed_time = end_time - start_time  
  
print(f'Elapsed time: {elapsed_time} seconds')

## Translating the date from Spanish / Italian / French to English 
- Allows us to convert to datetime format

In [22]:
from datetime import datetime, date 
pd.set_option('display.max_columns', None)
amazon2 = pd.DataFrame(all_reviews)

import re

month_translation = {
    'Spain': {
        'enero': 'January', 'febrero': 'February', 'marzo': 'March', 'abril': 'April', 
        'mayo': 'May', 'junio': 'June', 'julio': 'July', 'agosto': 'August', 
        'septiembre': 'September', 'octubre': 'October', 'noviembre': 'November', 'diciembre': 'December'
    },
    'France': {
        'janvier': 'January', 'février': 'February', 'mars': 'March', 'avril': 'April', 
        'mai': 'May', 'juin': 'June', 'juillet': 'July', 'août': 'August', 
        'septembre': 'September', 'octobre': 'October', 'novembre': 'November', 'décembre': 'December'
    },
    'Italy': {
        'gennaio': 'January', 'febbraio': 'February', 'marzo': 'March', 'aprile': 'April', 
        'maggio': 'May', 'giugno': 'June', 'luglio': 'July', 'agosto': 'August', 
        'settembre': 'September', 'ottobre': 'October', 'novembre': 'November', 'dicembre': 'December'
    }
}

# Function to translate dates
def translate_date(date_str, language):
    if pd.isna(date_str) or date_str.strip() == '':
        return None
    if language in month_translation:
        pattern1 = r'(\d{1,2}) de (\w+) de (\d{4})'
        pattern2 = r'(\d{1,2})\s+(?:de\s+)?(\w+)\s+(\d{4})'
        match1 = re.match(pattern1, date_str)
        match2 = re.match(pattern2, date_str)
        if match1:
            day, month, year = match1.groups()
            month = month_translation[language].get(month.lower(), month)
            return f"{day} {month.capitalize()} {year}"
        elif match2:
            day, month, year = match2.groups()
            month = month_translation[language].get(month.lower(), month)
            return f"{day} {month.capitalize()} {year}"
    return date_str
def translate_dates(row):
    country = row['Amazon site']
    if country in month_translation:
        row['Review date'] = translate_date(row['Review date'], country)
    return row
    
amazon2['Review date'] = amazon2['Review date'].astype(str)
amazon2 = amazon2.apply(translate_dates, axis=1)

# Function to convert multiple date formats
def convert_to_datetime(date_str):
    formats = ["%B %d, %Y", "%d %B %Y"]
    for fmt in formats:
        try:
            return datetime.strptime(date_str, fmt)
        except ValueError:
            continue
    return None

amazon2['Review date'] = amazon2['Review date'].apply(lambda x: convert_to_datetime(x) if x else None)

## Save Result

In [23]:
amazon2['Retailer']="Amazon"
# amazon2['Scraping Date'] = pd.to_datetime(date.today())
# amazon2['Review date'] = amazon2['Review date'].str.strip()
# amazon2['Review date'] = pd.to_datetime(amazon2['Review date'], infer_datetime_format=True, errors='coerce')

def extract_title(title):
    if isinstance(title, str):
        match = re.match(r'.*?\n(.*)', title)
        if match:
            return match.group(1)
    return title
amazon2['Review title'] = amazon2['Review title'].apply(extract_title)

# amazon2['Review title'] = amazon2['Review title'].str.replace(r'^.*?\n', '')
# amazon2['Review title'] = amazon2['Review title'].str.strip()
# amazon2['Review title'] = amazon2['Review title'].str.extract(r'out of 5 stars\n(.*)')
amazon2['HP Model Number'] = amazon2['Model'].str.extract(r'(\d+e?)')
# amazon2['People find helpful'] = amazon2['People find helpful'].str.extract(r'(\d*) people found this helpful')
  
selected_columns = ['Retailer', 'Amazon site', 'HP Model Number', 'Model', 'Review average', 'Review date', 'Review title', 'Review rating', 
                    'Review content', 'Review country', 'URL']
  
amazon_review = amazon2[selected_columns]  

amazon_review = amazon_review.drop_duplicates()

# Remove rows with no reviews and review scores
amazon_review['Review average'] = pd.to_numeric(amazon_review['Review average'], errors='coerce')
amazon_review = amazon_review[amazon_review['Review average'] != 0.0]

# Remove rows with repeated data
def remove_repeated(data):
    non_empty_rows = data.dropna(subset=['Review date', 'Review title', 'Review rating', 'Review content'], how='all')
    if not non_empty_rows.empty:
        # If there are non-empty rows, drop all rows with any empty field
       return non_empty_rows
    else:
        # If all rows have empty fields, keep only the first row
        return data.head(1)

# Apply the function to each model
amazon_review = amazon_review.groupby('Model').apply(remove_repeated).reset_index(drop=True)


date = datetime.today().strftime('%Y%m%d')  
amazon_review.to_excel(f'scraped_reviews/amazon_reviews_{date}_test1.xlsx')

# Best Buy

In [2]:
def get_review_bestbuy(url):
    extracted_reviews = []
    retry_count = 0
    header = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36 Edg/118.0.2088.61',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9',
        'Cache-Control': 'max-age=0',
        'Cookie': 'SID=5dd8d974-1010-4705-9db0-0091b9be90eb; bby_rdp=l; CTT=422cf77c62f741992b73b7eb194dd19d; intl_splash=false; intl_splash=false; vt=d36b7cc9-70f1-11ee-af65-0a4fc06e3e21; rxVisitor=169798943988975DRVD09AP9VHNKB488A7AMQ2ITCSNQ3; COM_TEST_FIX=2023-10-22T15%3A44%3A00.270Z; __gads=ID=6d604286666986e7:T=1697989449:RT=1697989449:S=ALNI_Mb_Z6tWUAT9d1smc0S2VYNtEXVnJQ; __gpi=UID=00000c6de2768122:T=1697989449:RT=1697989449:S=ALNI_MY8b96wWX_3ahxWOvsLcoQi2kpHIA; s_ecid=MCMID%7C51499273735922173403879288947271341352; AMCVS_F6301253512D2BDB0A490D45%40AdobeOrg=1; dtCookie=v_4_srv_5_sn_UKGS61LHKE95F58CKCJ5JTTUHNJV2N7D_app-3A1b02c17e3de73d2a_1_ol_0_perc_100000_mul_1; _cs_mk=0.5500628905410729_1697989446664; s_cc=true; AMCV_F6301253512D2BDB0A490D45%40AdobeOrg=1585540135%7CMCMID%7C51499273735922173403879288947271341352%7CMCAID%7CNONE%7CMCOPTOUT-1697996646s%7CNONE%7CMCAAMLH-1698594246%7C3%7CMCAAMB-1698594246%7Cj8Odv6LonN4r3an7LhD3WZrU1bUpAkFkkiY1ncBR96t2PTI%7CMCCIDH%7C1907712470%7CvVersion%7C4.4.0; aam_uuid=56460070521806806704392296716542884874; locDestZip=96939; locStoreId=1760; sc-location-v2=%7B%22meta%22%3A%7B%22CreatedAt%22%3A%222023-10-22T15%3A44%3A06.975Z%22%2C%22ModifiedAt%22%3A%222023-10-22T15%3A44%3A07.381Z%22%2C%22ExpiresAt%22%3A%222024-10-21T15%3A44%3A07.381Z%22%7D%2C%22value%22%3A%22%7B%5C%22physical%5C%22%3A%7B%5C%22zipCode%5C%22%3A%5C%2296939%5C%22%2C%5C%22source%5C%22%3A%5C%22G%5C%22%2C%5C%22captureTime%5C%22%3A%5C%222023-10-22T15%3A44%3A06.975Z%5C%22%7D%2C%5C%22destination%5C%22%3A%7B%5C%22zipCode%5C%22%3A%5C%2296939%5C%22%7D%2C%5C%22store%5C%22%3A%7B%5C%22storeId%5C%22%3A1760%2C%5C%22zipCode%5C%22%3A%5C%2299504%5C%22%2C%5C%22storeHydratedCaptureTime%5C%22%3A%5C%222023-10-22T15%3A44%3A07.380Z%5C%22%7D%7D%22%7D; __gsas=ID=43dc00dcffeab34e:T=1697989465:RT=1697989465:S=ALNI_MYLHkniZY8kqCiAFOeNu1jnR4mz0w; dtSa=-; cto_bundle=2D7FnF9ZMHJPQlFCbkdTMktUSFREZ2pVJTJGajJMRFFsd2lINnRNRkZxY0dFU1lqJTJCN0glMkZMU0FqRTR0UyUyRmZRa1FscDdyV0tQUTNZdzVBM1g2WkJHUENTUEdlaGtUdWtiZWU4allOYlc2dyUyRm1VeiUyRlVBZVZkdVRmSFElMkJZQ0ExRk9mZzZNV1VNd1ZYSXZ5RWZSeUFQdkJXZ3VxZzZJZyUzRCUzRA; blue-assist-banner-shown=true; _cs_c=1; _gcl_au=1.1.1372174147.1697989479; dtLatC=1; _abck=2025C1ED2DAE1BA19B91708C91F51C0F~0~YAAQHLQRYGhMakWLAQAAjTYLWgqyxn7G2wIoFoVC+4nrsT1cxJIaO1O5ytS58DrifnksxvYxu7oOIuZmBDszkeEGLUk/7ekIvtGFO7u2yogmIcW17juPvPSDc1XdGYIVbijt6PbXvKVWeAB+8ZIF6voDPAwIN8H+QKpGl7va06mSquCsIXDORvQ1fz6MaHlKajkG/g9N8gGFlrsBxnMpRA0vk4b7Xv9obYx0wvld8KvntBNHHmpIs0djlSe17djNQz57X3JJHstt9/StCh7Jo00MTiV93eKEGVBoMzoq4+PxnTdsrKg5PkI1bneUzJMSGuV43ZaXWfbm7uJ5sVfxdvHl0uQOQUh7ClSLpjFxe7sR9F6ZRsJ1uTIjK2Ab7WfvjLZd5C8V7/qZhg/oMP3pF0Dt09LThXO7tonFOvt8UhAETsU0Hw6+K/m4mS0wH46V+5rfa+qmNcM=~-1~-1~-1; bm_sz=DC447A131B862AC781959292B401C641~YAAQHLQRYGlMakWLAQAAjTYLWhWe27kjreKQmsKd+a2iqr9yFDHU3maKKvHTexZicnoFjIsx0OiZ03lAbfGOl2IZo7UNsbeBjNT3emSu3sSR0HUl0ddFd8LjnFGqQISSIw7upSTqhbE/Ccdgbo842X0fWkxXLQCXe7eIC5cgVWU1GMRdWc34I/WgCiVwaRV2v6j3I93rIKuMA5dYvCv2yQykBKCPPN4sbyl8TEvfZ+XgvWuziGVpb4G+3OBohzrz8/j7ZnhXQ1U0WZARKye28p1zLuSDfDk4mInPZlvumI5oeG13Z+CjpYEKf7D5iAjzcWRGlsQ32gejCk7aPI6RC1dkVBh/DL00bGUor4wdKjruVwQNpz0v3hop17nvb4BKkQIqQAfEL6zMaGHLj9ycBq93U+2b2AXxNlcKMzEYuQ0cL/PuJIlwGBiqjQ==~4601667~3422276; dtPC=5$589618119_27h-vVFAAHBSMMCTLEHFRWVQLIOPDRRSURPFR-0e0; rxvt=1698031868366|1698030068366; _cs_id=3fe9d270-9876-ad3a-cae4-0084c344a27c.1697989478.5.1698030169.1698030169.1645469968.1732153478774; _cs_s=1.0.0.1698031970357; c2=pdp%3A%20ratingsreviews',  # Replace with the actual Cookie
        'Downlink': '10',
        'Dpr': '1',
        'Referer': url,
        'Sec-Ch-Ua': '"Google Chrome";v="117", "Not;A=Brand";v="8", "Chromium";v="117"',
        'Sec-Ch-Ua-Mobile': '?0',
        'Sec-Ch-Ua-Platform': '"Windows"',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'cross-site',
        'Sec-Fetch-User': '?1',
        'Upgrade-Insecure-Requests': '1'
    }

    response = requests.get(url, headers=header)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup 

def bestbuy_review(soup, url):    
    bestbuy = {}
    bestbuy_reviews = []   
    Model =  soup.find("h2", {"class": "heading-6 product-title mb-100"}).text
    Review_average = soup.find("span", {"class": "ugc-c-review-average font-weight-medium order-1"}).text
        
    review_session = soup.find_all("div", {"class": "review-item-content col-xs-12 col-md-9"})
    for item in review_session:    
        bestbuy = {    
             'Model': Model,
             'Review average': Review_average,
             'Review date': item.find("div", {"class": "posted-date-ownership disclaimer v-m-right-xxs"}).text.replace('Posted','')  
            ,'URL':url 
        }
        
        try:    
            bestbuy['Review title']  = item.find("h4", {"class": "c-section-title review-title heading-5 v-fw-medium"}).text  
        except AttributeError:    
            bestbuy['Review title']  = None
        # print(bestbuy['Review title'])    
        try:    
            bestbuy['Review rating']  = item.find("div", {"class": "c-ratings-reviews flex c-ratings-reviews-small align-items-center gap-50"}).text.replace(' out of 5 stars','').replace('Rated ','')  
        except AttributeError:    
            bestbuy['Review rating']  = None
            
        # try:    
        #     bestbuy['Review promotion']  = item.find("div", {"class": "body-copy-sm pt-50"}).text
        # except AttributeError:    
        #     bestbuy['Review promotion']  = None
            
        # try:    
        #     bestbuy['Review aggregation']  = item.find("p", {"class": "body-copy ugc-related-product"}).text
        # except AttributeError:    
        #     bestbuy['Review aggregation']  = None
            
        try:    
            bestbuy['Review content']  = item.find("div", {"class": "ugc-review-body"}).text  
        except AttributeError:    
            bestbuy['Review content']  = None
            
        # try:    
        #     bestbuy['Review Recommendation']  = item.find("div", {"class": "ugc-recommendation"}).text  
        # except AttributeError:    
        #     bestbuy['Review Recommendation']  = None
            
        # try:    
        #     network_badge  = item.find("div", {"class": "ugc-network-badge"})
        #     if network_badge:
        #         bestbuy['Seeding or not'] = network_badge.get("data-track")
        # except AttributeError:    
        #     bestbuy['Seeding or not']  = None
            
        # try:    
        #     bestbuy['People find helpful']  = item.find("button", {"data-track": "Helpful"}).text
        # except AttributeError:    
        #     bestbuy['People find helpful']  = None
            
        # try:    
        #     bestbuy['People find unhelpful']  = item.find("button", {"data-track": "Unhelpful"}).text
        # except AttributeError:    
        #     bestbuy['People find unhelpful']  = None

  
        bestbuy_reviews.append(bestbuy)    
    
  
    return bestbuy_reviews 

## Links

In [3]:
path = r"Review text scrape URL.xlsx"
sheets = 'Bestbuy'
bestbuy_url = pd.read_excel(path, sheet_name = sheets)
urls = bestbuy_url['URL'].to_list()
url_list = []
for value in urls:
    if pd.notna(value):
        url_list.append(value)
url_list
# url_list = ['https://www.bestbuy.com/site/reviews/hp-936-standard-capacity-ink-cartridge-cyan/6562271?variant=A',
#  'https://www.bestbuy.com/site/reviews/hp-936-standard-capacity-ink-cartridge-black/6562273?variant=A']

['https://www.bestbuy.com/site/reviews/hp-936-standard-capacity-ink-cartridge-cyan/6562271?variant=A',
 'https://www.bestbuy.com/site/reviews/hp-936-standard-capacity-ink-cartridge-black/6562273?variant=A',
 'https://www.bestbuy.com/site/reviews/hp-936-4-pack-standard-capacity-ink-cartridges-black-magenta-yellow-cyan/6562274?variant=A',
 'https://www.bestbuy.com/site/reviews/hp-923-standard-capacity-ink-cartridge-cyan/6562093?variant=A',
 'https://www.bestbuy.com/site/reviews/hp-923-standard-capacity-ink-cartridge-magenta/6562092?variant=A',
 'https://www.bestbuy.com/site/reviews/hp-923-standard-capacity-ink-cartridge-yellow/6562094?variant=A',
 'https://www.bestbuy.com/site/reviews/hp-923-standard-capacity-ink-cartridge-black/6562091?variant=A',
 'https://www.bestbuy.com/site/reviews/hp-923-4-pack-standard-capacity-ink-cartridges-black-magenta-yellow-cyan/6562090?variant=A',
 'https://www.bestbuy.com/site/reviews/hp-923e-evomore-ink-cartridge-cyan/6578588?variant=A',
 'https://www.bes

## Scrape Reviews

In [ ]:
bestbuy_reviews = []    
  
for link in url_list:    
    print(link)  
    # print("Total review",len(bestbuy_reviews))    
    should_continue = True  
    for x in range(1, 1000):  
        if not should_continue:  
            break   
        while True:
            try:
                url = f'{link}&page={x}'  
                soup = get_review_bestbuy(url)  
                # print(f'Getting page: {x}') 
                reviews = bestbuy_review(soup, url)  
                print(f'Extracted reviews on page {x}: {len(reviews)}')    

                bestbuy_reviews.extend(reviews)    
                
                next_page_link = soup.find("li", {"class": "page next"})  # Note: Use lowercase "true" for attribute value
                # print(next_page_link)
                if next_page_link is None:
                    should_continue = False
                    # print('No more pages left')
                    break 
                else:   
                    break
        
            except Exception as e:  
                        print(f"Error encountered: {e}. Retrying in 3 seconds...")  
                        time.sleep(3) 

## Save Result

In [ ]:
from datetime import datetime, date  
pd.set_option('display.max_columns', None)
bestbuy_review = pd.DataFrame(bestbuy_reviews)
bestbuy_review['Retailer']="bestbuy"
bestbuy_review['Scraping Date'] = date.today().strftime('%Y-%m-%d')
bestbuy_review['HP Model Number'] = bestbuy_review['Model'].str.extract(r'(\d+e*)')
# bestbuy_review['People find helpful'] = bestbuy_review['People find helpful'].str.extract(r'(\d+)').astype(int)
# bestbuy_review['People find unhelpful'] = bestbuy_review['People find unhelpful'].str.extract(r'(\d+)').astype(int)

new_column_order = ['Retailer', 'HP Model Number', 'Model', 'Review date', 'Review title', 'Review rating', 'Review content', 'Scraping Date', 'URL']
bestbuy_review = bestbuy_review[new_column_order]

In [ ]:
date = datetime.today().strftime('%Y%m%d')  
bestbuy_review.to_excel(f'scraped_reviews/bestbuy_reviews_{date}.xlsx')

# Staples

In [5]:
def get_soup_staple(url):   
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    print(url)
    driver.get(url)
    content = driver.page_source
    soup = BeautifulSoup(content, 'html.parser') 
    
    driver.quit()
    return soup  
  
def staple_review(soup, url):    
    staple = {}
    staple_reviews = []   
    try:
        model = soup.find("a", attrs={"class": "list-tile__product_title"}).string.strip()   
    except AttributeError: 
        model = soup.title.text.replace("Staples Customer Reviews for ","")   
    try: 
        review_average = soup.find("div", attrs={"class": "AverageRating__ratingCircleSection"}).text
    except AttributeError:
        review_average = soup.find("div", attrs={"class": "sc-14dq9fm-4 chBfYj"}).text
        
    review_session = soup.find_all("div", {"class": "ReviewList__reviewSection"})
    
    
    for item in review_session:    
        staple = {    
            'Model': model,    
            'Review average' : review_average,
            'Review date': item.find("div", {"class": "ReviewDate__reviewDate"}).text.replace('Posted',''),     
            "Review content": item.find("div", {"class": "ReviewDescription__reviewDescription"}).text,  
            "URL" : url  
        }
        
        try:    
            staple['Review title']  = item.find("div", {"class": "ReviewTitle__reviewTitle"}).text  
        except AttributeError:    
            staple['Review title']  = None
        # print(staple['Review title'])
            
        # try:    
        #     staple['Response name']  = item.find("div", {"class": "ReviewResponse__responseUser"}).text  
        # except AttributeError:    
        #     staple['Response name']  = None
            
        # try:    
        #     staple['Response text']  = item.find("div", {"class": "ReviewResponse__responseText"}).text  
        # except AttributeError:    
        #     staple['Response text']  = None
            
        # try:    
        #     staple['Response date']  = item.find("div", {"class": "ReviewResponse__responderDetail"}).text  
        # except AttributeError:    
        #     staple['Response date']  = None
            
        # try:
        #     staple['Seeding or not'] = item.find("div", {"class":"ReviewBadge__incentivized"}).text
        # except AttributeError:
        #     staple['Seeding or not'] = None
        
        # try:    
        #     staple["Verified Purchase or not"] = item.find("div", {"class": "ReviewBadge__badge"}).text  
        # except AttributeError:    
        #     staple["Verified Purchase or not"] = None    
  
        # try:      
        #     staple["Review name"] = item.find("div", {"class": "ReviewUser__reviewUser"}).text  
        # except AttributeError:        
        #     staple["Review name"] = None  
  
        # try:    
        #     staple["People find helpful"] = item.find("span", {"class": "ThumbsToggle__sr_only"}).text.replace('up votes ','')  
        # except AttributeError:    
        #     staple["People find helpful"] = None  
            
        # try:    
        #     staple["People find unhelpful"] = item.find("span", {"class": "ThumbsToggle__sr_only"}).text.replace('down votes ','')  
        # except AttributeError:    
        #     staple["People find unhelpful"] = None  
            
        try:
            staple['Syndicated source'] = item.find("div", {"class": "ReviewBadge__reviewedAtText"}).string
        except AttributeError:   
            staple['Syndicated source'] = None 
            
        review_star_sections = item.find_all("div", {"class": "ReviewRow__reviewStarSection"})
        for star in review_star_sections:  
            rating_div = star.find("div", {"class": re.compile(r'\d_star')})  
        if rating_div:  
            staple["Review rating"] = re.search(r'(\d)_star', rating_div['class'][0]).group(1)  

  
        staple_reviews.append(staple)    
    
  
    return staple_reviews 

## Links

In [6]:
path =  r"Review text scrape URL.xlsx"
sheets = 'Staples'
staple_url = pd.read_excel(path, sheet_name = sheets)
urls = staple_url['URL'].to_list()

url_list = []
for value in urls:
    if pd.notna(value):
        url_list.append(value)
url_list
# url_list = ['https://www.staples.com/ptd/review/24590167']

['https://www.staples.com/ptd/review/24582881',
 'https://www.staples.com/ptd/review/24582882',
 'https://www.staples.com/ptd/review/24582883',
 'https://www.staples.com/ptd/review/24582884',
 'https://www.staples.com/ptd/review/24582885',
 'https://www.staples.com/ptd/review/24582886',
 'https://www.staples.com/ptd/review/24582887',
 'https://www.staples.com/ptd/review/24582888',
 'https://www.staples.com/ptd/review/24582889',
 'https://www.staples.com/ptd/review/24582890',
 'https://www.staples.com/ptd/review/24590160',
 'https://www.staples.com/ptd/review/24590161',
 'https://www.staples.com/ptd/review/24590162',
 'https://www.staples.com/ptd/review/24590163',
 'https://www.staples.com/ptd/review/24590164',
 'https://www.staples.com/ptd/review/24590165',
 'https://www.staples.com/ptd/review/24590166',
 'https://www.staples.com/ptd/review/24590167',
 'https://www.staples.com/ptd/review/24599304',
 'https://www.staples.com/ptd/review/24599307',
 'https://www.staples.com/ptd/review/245

## Scrape Reviews

In [ ]:
staple_reviews = []    
  
for link in url_list:    
    print(link)  
    should_continue = True  
    for x in range(1, 500):  
        if not should_continue:  
            break   
        while True:
            try:
                url = f'{link}/?page={x}'  
                soup = get_soup_staple(url)  
                # print(f'Getting page: {x}')    
                reviews = staple_review(soup, url)   
                print(f'Extracted reviews on page {x}: {len(reviews)}')    

                staple_reviews.extend(reviews)    
#                 print("Total review",len(staple_reviews))    


                next_page_link = soup.find("a", {"aria-label": "Next page of results"})       
                # print(next_page_link)
                if next_page_link is not None:
                    tabindex = next_page_link.get("tabindex")      
                    if tabindex == "-1": 
                        should_continue = False 
                        print('No more pages left')      
                        break
                    else:
                        break
                else: 
                    should_continue = False 
                    break
        
            except Exception as e:  
                        print(f"Error encountered: {e}. Retrying in 3 seconds...")  
                        time.sleep(3) 

## Save Result

In [ ]:
from datetime import date, datetime  
pd.set_option('display.max_columns', None)
staple = pd.DataFrame(staple_reviews)
staple
staple['Retailer']="Staples"
# staple['Scraping Date'] = date.today().strftime('%Y-%m-%d')
staple['HP Model Number'] = staple['Model'].str.extract(r'(\d+e?)')

# staple['Review date'] = pd.to_datetime(staple['Review date'])
staple['Review date'] = pd.to_datetime(staple['Review date'].str.strip(), format='%b %d, %Y')
# staple['People find helpful'] = staple['People find helpful'].str.extract(r'(\d+)').astype(int)
# staple['People find unhelpful'] = staple['People find unhelpful'].str.extract(r'(\d+)').astype(int)

new_column_order = ['Retailer', 'HP Model Number', 'Model', 'Review date', 'Review title', 'Review rating', 'Review content', 
                    'Syndicated source', 'URL']
staple = staple[new_column_order]

date = datetime.today().strftime('%Y%m%d')  
staple.to_excel(f'scraped_reviews/staple_reviews_{date}.xlsx')

# Walmart 
## Run either method 1 or 2

## Method 1: Use API
Apply for API: https://api.scrapingdog.com/login (need to register account, one account has 1K times for scraping)

In [8]:
def get_soup_walmart(url, max_retries=5):  
    extracted_reviews = []  
    retry_count = 0  

    # headers = {
    # 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    # 'Accept-Language': 'en-US,en;q=0.5',
    # 'Referer': 'https://www.google.com/',
    # }
    while retry_count < max_retries:  
        response = requests.get(url)  
        soup = BeautifulSoup(response.text, 'html.parser')  
        # html_content = response.text   
        return soup
    # while retry_count < max_retries:  
    #     try:
    #         response = requests.get(url, headers=headers)  
    #         if response.status_code == 200:
    #             soup = BeautifulSoup(response.text, 'html.parser')  
    #             return soup
    #         else:
    #             print(f"Error: Received response code {response.status_code}")
    #     # html_content = response.text   
    #     # print(response)
    #     except requests.RequestException as e:
    #         print(f"Request failed: {e}")
    # return None

def walmart_review(soup, url):    
    parsed_url = urlparse(url)
    url_param = parse_qs(parsed_url.query).get('url', [''])[0]
    walmart = {}  
    walmart_reviews = []  
    try:
        Model = soup.find("a", {"class": "w_x7ug f6 dark-gray"}).text.replace('Back to ', '').strip()
    except AttributeError:
        Model = None
    # print(Model)
    try:
        Review_average = soup.find("span", {"class": "f-headline b"}).text.replace(' out of 5', '').strip()
    except AttributeError:
        Review_average = None
    # print(Review_average)
    
    review_section = soup.find_all("li", {"class": "dib w-100 mb3"})
    
    for item in review_section:    
        walmart = {    
            'Model': Model,    
            'Review average' : Review_average,
            # 'Review date': item.find("div", {"class": "f7 gray mt1"}).text,     
            # "Review Content": item.find("div", {"class": "f6 mid-gray lh-copy"}).text,  
            "URL" : url
        }
        try:    
            walmart['Review title']  = item.find("h3", {"class": "w_kV33 w_Sl3f w_mvVb f5 b"}).text  

        except AttributeError:    
            walmart['Review title']  = None
        print(walmart['Review title'])
        
        try:    
            walmart['Review rating']  = item.find("span", {"class": "w_iUH7"}).text.replace(' out of 5 stars review','')

        except AttributeError:    
            try:
                walmart['Review rating']  = item.find("div", {"class": "flex flex-grow-1"}).text.replace(' out of 5 stars review','')
            except:
                walmart['Review rating']  = None
        print(walmart['Review rating'])
        try: 
            walmart['Review content'] = item.find("span", {"class": "tl-m mb3 db-m"}).text
        except AttributeError:
            walmart['Review content'] = None

        try: 
            walmart['Review date'] = item.find("div", {"class": "f7 gray mt1"}).text
        except AttributeError:
            walmart['Review date'] = None
            
        # try:
        #     walmart['Incentivized review'] = item.find("span", {"class": "w_VbBP w_0FMU w_I_19 mb2 mr2"}).text 
        # except AttributeError:    
        #     walmart['Incentivized review']  = None

        # try:
        #     walmart['Verified Purchase or not'] = item.find("span", {"class": "green b mr1"}).text 
        # except AttributeError:    
        #     try: 
        #         walmart['Verified Purchase or not'] = item.find("div", {"class": "pl2 f7 self-center"}).text
        #     except:
        #         walmart['Verified Purchase or not']  = None
        walmart_reviews.append(walmart)   
    print(walmart_reviews)
    return walmart_reviews 


## Links

In [9]:
path = r"Review text scrape URL.xlsx"
sheets = "Walmart"
url = pd.read_excel(path, sheet_name = sheets)
all_list = url['URL'].to_list()
link_list = []
for value in all_list:
    if pd.notna(value):
        link_list.append(value)
link_list

## Scrape Reviews

In [ ]:
api_key = '665ed06172077d78c90a593c' # Replace with your api key

walmart_reviews = []    
    
for link in link_list:    
    print(link)  
    should_continue = True 
    for x in range(1, 500):  
        if not should_continue:  
            break   
        while True:
            try:
                url = f"https://api.scrapingdog.com/scrape?api_key={api_key}&url={link}/?page={x}"
                # url = f"https://api.scrapingdog.com/scrape?api_key={api_key}&url={link}/?page=3"
                soup = get_soup_walmart(url)
                # print(soup)
                # print('yes')
                reviews = walmart_review(soup, url)   
                # print('no')
                # print(reviews)
                print(f'Extracted reviews on page {x}: {len(reviews)}')  

                walmart_reviews.extend(reviews)

                next_page_link = soup.find("a", {"aria-label": "Next Page"})       
                # print(next_page_link)

                if next_page_link is None:
                    should_continue = False 
                    # print('No more pages left')      
                    break  
                else:   
                    break

            except Exception:  
                print(f"Error encountered. Retrying in 3 seconds...")  
                time.sleep(3)

## Method 2: Selenium

In [11]:
def get_soup(url):   
    # r = requests.get('http://localhost:8050/render.html', params={'url': url, 'wait': 2})  
    # soup = BeautifulSoup(r.text, 'html.parser')  
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    driver.get(url)
    content = driver.page_source
    soup = BeautifulSoup(content, 'html.parser')     
    driver.quit()
    # print(soup)
    return soup  

def walmart_review(soup, url):    
    walmart = {}  
    walmart_reviews = []  
    
    Model = soup.find("a", {"class": "w_x7ug f6 dark-gray"}).text.replace('Back to ', '').strip()
    # print("Model: ", Model) 
    Review_average = soup.find("span", {"class": "f-headline b"}).text.replace(' out of 5', '')
    # print(Review_average)
    review_section = soup.find_all("li", {"class": "dib w-100 mb3"})
    # print(review_section)
    for item in review_section:    
        walmart = {    
            'Model': Model,    
            'Review average' : Review_average,  
            # "Review Content": item.find("div", {"class": "f6 mid-gray lh-copy"}).text,  
            "URL" : url
        }
        try:    
            walmart['Review title']  = item.find("h3", {"class": "w_kV33 w_Sl3f w_mvVb f5 b"}).text  
        except AttributeError:    
            walmart['Review title']  = None
        # print(walmart['Review title'])
        
        try:    
            walmart['Review rating']  = item.find("span", {"class": "w_iUH7"}).text.replace(' out of 5 stars review','')
        except AttributeError:    
            try:
                walmart['Review rating']  = item.find("div", {"class": "flex flex-grow-1"}).text.replace(' out of 5 stars review','')
            except:
                walmart['Review rating']  = None
        # print(walmart['Review rating'])
        try: 
            walmart['Review content'] = item.find("span", {"class": "tl-m mb3 db-m"}).text
        except AttributeError:
            walmart['Review content'] = None
        try: 
            walmart['Review date'] = item.find("div", {"class": "f7 gray mt1"}).text
        except AttributeError:
            walmart['Review date'] = None
            
        # try:
        #     walmart['Incentivized review'] = item.find("span", {"class": "w_VbBP w_0FMU w_I_19 mb2 mr2"}).text 
        # except AttributeError:    
        #     walmart['Incentivized review']  = None

        # try:
        #     walmart['Verified Purchase or not'] = item.find("span", {"class": "green b mr1"}).text 
        # except AttributeError:    
        #     try: 
        #         walmart['Verified Purchase or not'] = item.find("div", {"class": "pl2 f7 self-center"}).text
        #     except:
        #         walmart['Verified Purchase or not']  = None

        walmart_reviews.append(walmart)    
    # print(walmart_reviews)
    return walmart_reviews 


## Links

In [34]:
path = r"Review text scrape URL.xlsx"
sheets = "Walmart"
url = pd.read_excel(path, sheet_name = sheets)
all_list = url['URL'].to_list()
link_list = []
for value in all_list:
    if pd.notna(value):
        link_list.append(value)
link_list

## Scrape Reviews

In [ ]:
walmart_reviews = []    
    
for link in link_list:    
    print(link)  
    should_continue = True 
    for x in range(1, 500):  
        if not should_continue:  
            break   
        while True:
            try:
                url = f"{link}?page={x}"
                soup = get_soup(url)  
                # print(soup)
                reviews = walmart_review(soup, url)   
                print(reviews)

                print(f'Extracted reviews on page {x}: {len(reviews)}')  

                walmart_reviews.extend(reviews)

                next_page_link = soup.find("a", {"aria-label": "Next Page"})       
                # print(next_page_link)

                if next_page_link is None:
                    should_continue = False 
                    # print('No more pages left')      
                    break  
                else:
                    break

            except Exception:  
                print(f"Error encountered. Retrying in 3 seconds...")  
                time.sleep(5)

## Save Result

In [8]:
from datetime import datetime, date  
pd.set_option('display.max_columns', None)
walmart = pd.DataFrame(walmart_reviews)
walmart['Retailer']= "Walmart"
walmart['HP Model Number'] = walmart['Model'].str.extract(r'(\d+[e]*)', expand=False)
walmart['Scraping Date'] = date.today().strftime('%Y-%m-%d')
walmart['Review date'] = pd.to_datetime(walmart['Review date'].str.strip(), format='%m/%d/%Y')

new_column_order = ['Retailer', 'HP Model Number', 'Model', 'Review date', 'Review title', 'Review rating', 'Review content', 'URL']

walmart = walmart[new_column_order]

date = datetime.today().strftime('%Y%m%d')  
walmart.to_excel(f'scraped_reviews/walmart_reviews_{date}.xlsx')

## HP 

### Products are scraped using SKU/Product ID by Bazaarvoice API

In [13]:
import math
import time

# Import product number that needs to scrape from excel 
path = r'Review text scrape URL.xlsx'
sheet = 'HP'
hp = pd.read_excel(path, sheet_name=sheet)
skus = hp['SKU'].to_list()

sku_list = []
for value in skus:
    if pd.notna(value):
        sku_list.append(value)
sku_list

In [14]:
def hp_review(sku):
    
    api_url = 'https://api.bazaarvoice.com/data/reviews.json'

#Input own pass key
    params = {
        'resource': 'reviews',
        'action': 'REVIEWS_N_STATS',
        'filter': f'productid:eq:{sku}',
        'include': 'authors,products,comments',
        'limit': 100,
        'offset': 0,
        'sort': 'submissiontime:desc',
        'passkey': '',
        'apiversion': '5.5',
        'displaycode': '8843-en_us'
    }

    response = requests.get(api_url, params=params)
    # if response.status_code != 200:
    #     print(f'Error: status code {response.status_code}')
    #     return pd.DataFrame()
    if response.status_code == 200:
        data = json.loads(response.text)

    limit = 100
    no_batch = math.ceil(data['TotalResults']/limit)
    print('Total review',data['TotalResults'])

    df = pd.DataFrame()
    for x in range(0, no_batch):
        #Input own pass key
        offset = x*limit
#         print('Offset',offset)
        params = {
        'resource': 'reviews',
        'action': 'REVIEWS_N_STATS',
        'filter': f'productid:eq:{sku}',
        'include': 'authors,products,comments',
        'limit': 100,
        'offset': offset,
        'sort': 'submissiontime:desc',
        'passkey': '',
        'apiversion': '5.5',
        'displaycode': '8843-en_us'
        }

        response = requests.get(api_url, params=params)
        if response.status_code == 200:
            data = json.loads(response.text)
        else:
            print('status code != 200')

        results_data = data['Results']
        df_temp = pd.DataFrame(results_data)
        df = pd.concat([df, df_temp], axis=0)
        time.sleep(3)
    if not df.empty:
        df['SyndicationSource_Name'] = df.apply(lambda row: row['SyndicationSource'].get('Name') if row['IsSyndicated'] else None, axis=1)
        required_columns = ['OriginalProductName', 'SubmissionTime', 'Title', 'Rating', 'ReviewText', 'SyndicationSource_Name']
        available_columns = [col for col in required_columns if col in df.columns]
        df = df[available_columns]

        # Rename the columns
        new_column_names = {
            'OriginalProductName': 'Model',
            'Rating': 'Review rating',
            'SubmissionTime': 'Review date',
            'Title': 'Review title',
            'ReviewText': 'Review content',
            'SyndicationSource_Name': 'Syndicated source'
        }
        df = df.rename(columns=new_column_names)
        df['HP Model Number'] = df['Model'].str.extract(r'(\d+e?)')

        cols = df.columns.tolist()
        cols = ['HP Model Number'] + cols[:-1]  # Move 'HP Model Number' to the front
        df = df[cols]
    return df

In [ ]:
hp_reviews = pd.DataFrame()
for sku in sku_list:
    print('Get review', sku)
    data = hp_review(sku)
    # print(data)
    print('Review count',len(data))
    if data is not None:
        hp_reviews = pd.concat([hp_reviews,data],axis = 0)
        print('All Review count',len(hp_reviews))

In [ ]:
hp_reviews['Retailer']= "HP US"
hp_reviews['Review date'] = pd.to_datetime(hp_reviews['Review date'])
hp_reviews['Review date'] = hp_reviews['Review date'].dt.tz_localize(None)

new_column_order = ['Retailer', 'HP Model Number', 'Model', 'Review date', 'Review title', 'Review rating', 'Review content', 
                   'Syndicated source']

hp_reviews = hp_reviews[new_column_order]

from datetime import datetime, date  
date = datetime.today().strftime('%Y%m%d')  
hp_reviews.to_excel(f'scraped_reviews/HP_reviews_{date}.xlsx')

## Combine Results

In [ ]:
combined_df = pd.concat([amazon_review, bestbuy_review, staple, walmart, hp_reviews], ignore_index=True, sort=False)
combined_df

combined_df.to_excel(f'scraped_reviews/all_reviews_{date}.xlsx')